In [14]:
import keras
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout ,Input
from keras.models import Model, load_model
from keras import optimizers 
from keras.callbacks import TensorBoard,ModelCheckpoint
import argparse
from time import time
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers.convolutional import ZeroPadding2D
from keras import metrics
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
vgg16_model = keras.applications.vgg16.VGG16(input_shape=(224,224,3))

In [3]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
for layer in vgg16_model.layers[1:-3]:
  model.add(layer)

In [4]:
for layer in model.layers:
  layer.trainable = False

In [5]:
model.add(Dense(4,activation='softmax'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        multiple                  1792      
_________________________________________________________________
block1_conv2 (Conv2D)        multiple                  36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
block2_conv1 (Conv2D)        multiple                  73856     
_________________________________________________________________
block2_conv2 (Conv2D)        multiple                  147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
model.compile(optimizer = 'rmsprop',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [8]:
train_path = '/home/dj/Desktop/Datasets/at_200x/train'
test_path = '/home/dj/Desktop/Datasets/at_200x/val'

In [15]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>0.98):
      print("\nReached 98% accuracy so cancelling training!")
      self.model.stop_training = True

      

In [16]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path,target_size=(224,224),shuffle=True,class_mode = 'categorical',batch_size = 20)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,target_size=(224,224),shuffle=True,class_mode = 'categorical',batch_size = 20)

Found 948 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [17]:
callbacks = myCallback()
model.fit_generator(train_batches,steps_per_epoch=10,validation_data = test_batches,epochs=40,validation_steps=50,verbose=1,callbacks=[callbacks])

Epoch 1/40
10/10 [==============================] - 12s 1s/step - loss: 0.0365 - acc: 0.9900 - val_loss: 0.1123 - val_acc: 0.9480
Epoch 2/40
10/10 [==============================] - 11s 1s/step - loss: 0.0234 - acc: 0.9900 - val_loss: 0.0554 - val_acc: 0.9750
Epoch 3/40
10/10 [==============================] - 11s 1s/step - loss: 0.0813 - acc: 0.9750 - val_loss: 0.1762 - val_acc: 0.9360
Epoch 4/40
10/10 [==============================] - 11s 1s/step - loss: 0.1503 - acc: 0.9450 - val_loss: 0.2376 - val_acc: 0.9140
Epoch 5/40
10/10 [==============================] - 11s 1s/step - loss: 0.0324 - acc: 0.9849 - val_loss: 0.0532 - val_acc: 0.9760
Epoch 6/40
10/10 [==============================] - 11s 1s/step - loss: 0.0485 - acc: 0.9850 - val_loss: 0.1410 - val_acc: 0.9490
Epoch 7/40
10/10 [==============================] - 11s 1s/step - loss: 0.0146 - acc: 0.9950 - val_loss: 0.0482 - val_acc: 0.9880
Epoch 8/40
10/10 [==============================] - 11s 1s/step - loss: 0.1516 - acc: 0.95

In [18]:
model.save('bgmt_1_0.96(200x).h5')

In [40]:
import numpy as np
from keras.preprocessing import image

 
# predicting images
path = '/home/dj/Desktop/testing_folder/turning_200x1.jpg'
img = image.load_img(path, target_size=(224,224,3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])/255
classes = model.predict(images, batch_size=10)
print(classes)
categ = ['buffing','grinding','milling','turning']
print(categ[np.argmax(classes)])

[[  3.27414200e-05   2.55344480e-01   1.96115058e-02   7.25011289e-01]]
turning


In [45]:
from keras.models import load_model

In [46]:
new = load_model('bgmt_1_0.96(200x).h5')

In [85]:
keras.__version__


'2.2.4'

In [61]:
import numpy as np
from keras.preprocessing import image

 
# predicting images
path = '/home/dj/Desktop/Datasets/23-05-19/500x/buffing@500x/buffing_500x2.jpg'
img = image.load_img(path, target_size=(224,224,3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])/255
classes = new.predict(images, batch_size=10)
print(classes)
categ = ['buffing','grinding','milling','turning']
print(categ[np.argmax(classes)])

[[  1.00000000e+00   5.64410367e-12   5.73530201e-09   1.56116483e-12]]
buffing
